# Importações e funções

In [10]:
import pandas as pd
import socket
import requests
import dns.resolver
import time

# Função para obter o IP de um domínio
def get_ip(domain):
    try:
        return socket.gethostbyname(domain)
    except Exception as e:
        return None

# Função para obter a localização do IP usando a API ipinfo.io
def get_location(ip):
    if ip:
        try:
            response = requests.get(f'https://ipinfo.io/{ip}/json')
            data = response.json()
            loc = data.get('loc', '').split(',')
            latitude = loc[0] if len(loc) > 0 else ''
            longitude = loc[1] if len(loc) > 1 else ''
            return {
                'city': data.get('city', ''),
                'region': data.get('region', ''),
                'country': data.get('country', ''),
                'org': data.get('org', ''),
                'postal': data.get('postal', ''),
                'timezone': data.get('timezone', ''),
                'hostname': data.get('hostname', ''),
                'latitude': latitude,
                'longitude': longitude,
            }
        except Exception as e:
            return {
                'city': '',
                'region': '',
                'country': '',
                'org': '',
                'postal': '',
                'timezone': '',
                'hostname': '',
                'latitude': '',
                'longitude': '',
            }
    return {
        'city': '',
        'region': '',
        'country': '',
        'org': '',
        'postal': '',
        'timezone': '',
        'hostname': '',
        'latitude': '',
        'longitude': '',
    }

# Função para obter o DNS reverso de um IP
def get_reverse_dns(ip):
    if ip:
        try:
            return socket.gethostbyaddr(ip)[0]
        except Exception as e:
            return None
    return None

# Função para obter o número de registros DNS de um IP
def get_dns_count(ip):
    if ip:
        try:
            answers = dns.resolver.resolve(ip, 'A')
            return len(answers)
        except Exception as e:
            return 0
    return 0






# Ler dataset

In [12]:
# Carregar o dataset
df = pd.read_csv('ps.csv', delimiter=';')


C:\Users\ricar\AppData\Local\Temp\ipykernel_17608\199498343.py:2: DtypeWarning: Columns (55,56,57,58,59,60,61) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('ps.csv', delimiter=';')


label
 1.0    134849
 0.0     99762
-1.0      1184
Name: count, dtype: int64
Todas as linhas cumprem a condição.


C:\Users\ricar\AppData\Local\Temp\ipykernel_17608\199498343.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['label'].fillna(-1, inplace=True)


# Linhas em que a label não se encontra preenchida

In [13]:
# Substituir valores vazios na coluna "label" por -1
df['label'].fillna(-1, inplace=True)

# Contar os valores únicos na coluna "label"
label_counts = df['label'].value_counts()
print(label_counts)

# Encontrar os índices das linhas com valor -1 na coluna "label"
indices_to_drop = df[df['label'] == -1].index

# Remover as linhas com valor -1 na coluna "label"
df.drop(indices_to_drop, inplace=True)

C:\Users\ricar\AppData\Local\Temp\ipykernel_17608\2910599684.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['label'].fillna(-1, inplace=True)


label
 1.0    134849
 0.0     99762
-1.0      1184
Name: count, dtype: int64


# Reverse module DNS

In [ ]:
# Adicionar colunas para armazenar IP, DNS, Reverse DNS e número de DNS
df['IP'] = df['URL'].apply(lambda url: get_ip(url.split('//')[1]))
df['DNS'] = df['URL'].apply(lambda url: url.split('//')[1])
df['Reverse DNS'] = df['IP'].apply(get_reverse_dns)
df['n_dns'] = df['IP'].apply(get_dns_count)

# Adicionar contador para pausar a cada 100 registros
for i in range(0, len(df), 100):
    if i > 0 and i % 100 == 0:
        print(f"Processed {i} records. Sleeping for 30 seconds...")
        time.sleep(30)

    # Processar blocos de 100 registros
    chunk = df.iloc[i:i+100]
    
    # Obter informações de localização e adicionar colunas separadas
    location_data = chunk['IP'].apply(get_location)
    df.loc[i:i+100, 'City'] = location_data.apply(lambda loc: loc['city'])
    df.loc[i:i+100, 'Region'] = location_data.apply(lambda loc: loc['region'])
    df.loc[i:i+100, 'Country'] = location_data.apply(lambda loc: loc['country'])
    df.loc[i:i+100, 'Org'] = location_data.apply(lambda loc: loc['org'])
    df.loc[i:i+100, 'Postal'] = location_data.apply(lambda loc: loc['postal'])
    df.loc[i:i+100, 'Timezone'] = location_data.apply(lambda loc: loc['timezone'])
    df.loc[i:i+100, 'Hostname'] = location_data.apply(lambda loc: loc['hostname'])
    df.loc[i:i+100, 'Latitude'] = location_data.apply(lambda loc: loc['latitude'])
    df.loc[i:i+100, 'Longitude'] = location_data.apply(lambda loc: loc['longitude'])

    # Removing unnamed columns using drop function
df.drop(df.columns[df.columns.str.contains('Unnamed', case=False)], axis=1, inplace=True)

# Salvar o resultado em um novo arquivo CSV
df.to_csv('ps_with_ip_location_dns.csv', index=False)

# Exibir o DataFrame
print(df.head())